In [9]:
import sys
import numpy as np
from scipy.stats import multivariate_normal
import signal
import torch
import re
import pandas as pd
from torch.distributions.multivariate_normal import MultivariateNormal
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../SOGA-main/src')
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *

torch.set_default_dtype(torch.float64)
from fitness.soga_fitness_trueskills import *

In [10]:
model_name = 'mog1'

In [11]:
from os import getcwd, listdir, path, sep

file_path = path.join(getcwd(), "..", "results")
file_path = path.join(file_path, model_name)

# Find list of all runs contained in the specified folder.
runs = [run for run in listdir(file_path) if
        path.isdir(path.join(file_path, run))]

avg_fitness_5000 = []

# Iterate over all runs
for run in runs:
    # Get file name
    file_name = path.join(file_path, str(run), "best.txt")

    try:
        # Load in data
        file = open(file_name) 
    except:
        avg_fitness_5000.append(np.nan)
        continue
    flag = False
    for line in file.readlines():
        if 'Fitness on 5000 data' in line:
            flag = True
            continue
        if flag:
            avg_fitness_5000.append(float(line[:-2]))
            break

print('Average fitness of best found program on 5000 data: ', np.nanmean(avg_fitness_5000))
best_program_file = path.join(file_path, str(runs[np.nanargmax(avg_fitness_5000)]), "best.txt")
file = open(best_program_file)
text_bf = ''
flag = False
for line in file.readlines():
    if 'Phenotype' in line:
        flag = True
        continue
    if 'Genotype' in line:
        flag = False
        break
    if flag:
        text_bf += line + ' '

fitness_bf = likelihood_of_program_wrt_data(text_bf)

text_baseline = 'mu = gm([1, 20, 3]);\nsigma = gm([1,2,1]);\nx = sigma * gm([1,1,1]);\nx = x + mu;'

fitness_bl = likelihood_of_program_wrt_data(text_baseline)

print('Fitness of best found program: ', fitness_bf)
print('Fitness of baseline: ', fitness_bl)
print(text_bf) 

Average fitness of best found program on 5000 data:  -7.886998767439126
Fitness of best found program:  -6.988155421378471
Fitness of baseline:  -6.103008608975963
mu = gm([30, 19.4, 2], [0.4, 25.1, 13], [25.2, 26.2, 28.4], [19.2, 19.4, 2], [12.3, 9, 30.3]); 
 sigma = mu  - uniform([9, 15.1], 2); 
 x = mu + 2  * gm([19.4, 1, 0.9]);
 
 


In [12]:
# Computes SOGA distributions for best found and baseline

soga_bf = preprocess_program(text_bf)
compiledText=compile2SOGA_text(soga_bf)
cfg = produce_cfg_text(compiledText)
output_dist = start_SOGA(cfg)

soga_baseline = preprocess_program(text_baseline)
compiledText_baseline=compile2SOGA_text(soga_baseline)
cfg_baseline = produce_cfg_text(compiledText_baseline)
output_dist_baseline = start_SOGA(cfg_baseline)

In [13]:
# Generates data from original data generating process 

from fitness import data_generating_process as dgp  

data = dgp.generate_dataset(model_name, 100)
data_var_list, dependencies, weights = dgp.get_vars(model_name)

In [14]:
def get_mixture_pdf(point, var, dist):
    pdf = 0
    #indexes = {element: [index for index, value in enumerate(output_dist.var_list) if value == element] for element in data_var_list}
    for i in range(dist.gm.n_comp()):
        marginal_means = []
        covariance_index = []
        #for element, index_list in indexes.items():
        marginal_means.append(dist.gm.mu[i][dist.var_list.index(var)])
        covariance_index.append(dist.var_list.index(var))

        marginal_covariance_matrix = dist.gm.sigma[i][np.ix_(covariance_index,covariance_index)]
        
        pdf+= dist.gm.pi[i] * multivariate_normal.pdf(point, mean = marginal_means, cov = marginal_covariance_matrix, allow_singular=True)

    return pdf
        

In [ ]:
import matplotlib
matplotlib.use('TkAgg') 

import matplotlib.pyplot as plt
import numpy as np
import torch
import matplotlib.cm as cm


# Define the range for the uniform distribution
x = np.linspace(-10, 10, 500)
x = torch.tensor(x)

# Assuming data_var_list is defined
#data_var_list = ['a', 'b']  # Example list, replace with your actual list
colormap = cm.get_cmap('viridis', len(data_var_list))
colors = [colormap(i) for i in range(len(data_var_list))]


# Create a figure with subplots
fig, axs = plt.subplots(1, len(data_var_list), figsize=(14, 6))

# If there's only one element in data_var_list, axs will not be an array, so we need to handle that case
if len(data_var_list) == 1:
    axs = [axs]

# Loop through each element in data_var_list and create a subplot
for i, var in enumerate(data_var_list):
    color = colors[i]
    # select the x axis
    idx = output_dist.var_list.index(var)
    mean = output_dist.gm.mean()[idx]
    std = np.sqrt(output_dist.gm.cov()[idx, idx])
    x = np.linspace(mean-2*std, mean+2*std, 500)
    x = torch.tensor(x)
    # plots
    axs[i].plot(x, [get_mixture_pdf([xi], var, output_dist_baseline) for xi in x], label='Program', color='#333366')   
    axs[i].hist([pair[i] for pair in data], bins=50, density=True, alpha=0.5, color=color, label='Data')
    axs[i].set_title(f'Marginal distribution of "{var}"')
    axs[i].set_xlabel(f"{var}")
    axs[i].set_ylabel("Probability Density")
    axs[i].spines['right'].set_visible(False)
    axs[i].spines['top'].set_visible(False)
    axs[i].grid(True)
    axs[i].legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

/var/folders/wj/s0d4084s7rz93njmy26_wwnw0000gn/T/ipykernel_7951/1418629202.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('viridis', len(data_var_list))


FloatingPointError: underflow encountered in exp

In [ ]:
#3D image of the joint distribution
from mpl_toolkits.mplot3d import Axes3D


def get_mixture_pdf1(x, y):
    pdf = 0
    indexes = {element: [index for index, value in enumerate(output_dist.var_list) if value == element] for element in data_var_list}
    for i in range(output_dist.gm.n_comp()):
        marginal_means = []
        covariance_index = []
        for element, index_list in indexes.items():
            marginal_means.append(output_dist.gm.mu[i][index_list][0])
            covariance_index.append(index_list[0])

        marginal_covariance_matrix = output_dist.gm.sigma[i][np.ix_(covariance_index,covariance_index)]
        
        pdf += output_dist.gm.pi[i] * multivariate_normal.pdf([x, y], mean = marginal_means, cov = marginal_covariance_matrix, allow_singular=True)

    return pdf

def get_mixture_pdf(point, data_var_list = data_var_list):
    pdf = 0
    indexes = {element: [index for index, value in enumerate(output_dist.var_list) if value == element] for element in data_var_list}
    for i in range(output_dist.gm.n_comp()):
        marginal_means = []
        covariance_index = []
        for element, index_list in indexes.items():
            marginal_means.append(output_dist.gm.mu[i][index_list][0])
            covariance_index.append(index_list[0])

        marginal_covariance_matrix = output_dist.gm.sigma[i][np.ix_(covariance_index,covariance_index)]
        
        pdf+= + output_dist.gm.pi[i] * multivariate_normal.pdf(point, mean = marginal_means, cov = marginal_covariance_matrix, allow_singular=True)

    return pdf
        


x, y = np.mgrid[-10:10:.01, 0:15:.01]
pos = np.dstack((x, y))
#rv = multivariate_normal([0.5, -0.2], [[2.0, 0.3], [0.3, 0.5]])
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(x, y, get_mixture_pdf(pos, data_var_list=['a', 'b']))
ax2.scatter([pair[0] for pair in data], [pair[1] for pair in data], color = 'red', alpha = 0.7, s = 0.1, label = 'Data points')
ax2.set_title("Joint Distribution of the generated program")
ax2.set_xlabel("a")
ax2.set_ylabel("b")
ax2.legend()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from scipy.stats import multivariate_normal
var = multivariate_normal(mean=[0,0], cov=[[100000000,0],[0,0]], allow_singular=True)
var.pdf([1,1])

3.98942278406721e-05